We will be using pandas and sqlalchemy in order to read the taxi dataset from the <a href="http:\
//www.nyc.gov/html/tlc/html/about/trip_record_data.shtml">nyc.gov</a>.
Pandas is a package for R-like dataframes in python, which has great input-output (I/O).
You can find documentation on pandas I/O <a href="http://pandas.pydata.org/pandas-docs/stable/io\
.html">here</a>.
First, let's import what we'll need.

In [23]:
from sqlalchemy import create_engine
import pandas as pd
from pandas import io

Let's start by reading some of the csv using iterators in read_csv.

In [36]:
csvfile = "https://storage.googleapis.com/tlc-trip-data/2015/yellow_tripdata_2015-01.csv"

In [45]:
yellow_df = pd.read_csv(csvfile, iterator=True, chunksize = 10000)

engine = create_engine('sqlite:///taxi.sqlite')

The yellow_df is actually a readtextfile object, because we said iterator=True.  This means that it is iterable (we use it in a for below) and not a dataframe.  We have to use the next to get the next 1000 entries in the csv which is then a dataframe.

In [43]:
tablecur = yellow_df.next()
tablecur.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


We initialize the sql table 'yellow' below with the first 1000 lines.  We check to make sure that this matches what we have above.

In [46]:
tablecur.to_sql('yellow',engine)

In [47]:
io.sql.read_sql_table('yellow',engine).head()

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


Next we have to iterate over the remaining csv file and insert it into the sql table.  To insert we have to use the if_exists='append' arg.

In [50]:
for tablecur in yellow_df:
    tablecur.to_sql('yellow',engine,if_exists='append')

CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

Now, the first csv file is in the database in the local directory under taxi.sqlite.

TODO: append all of the csv files in this way by iterating over all of the urls of the CSVs (there is probably a pattern) and appending to the yellow table with the above line.  This will make a LARGE sqlite document.

In [53]:
io.sql.read_sql_query('SELECT * FROM yellow WHERE fare_amount=9.5',engine).head()

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
1,42,2,2015-01-25 00:13:06,2015-01-25 00:23:26,1,2.01,-73.948685,40.782360,1,N,-73.957085,40.801991,1,9.5,0.5,0.5,2.16,0.0,0.3,12.96
2,55,2,2015-01-25 00:13:09,2015-01-25 00:22:51,2,2.22,-73.992531,40.724461,1,N,-74.010895,40.703266,1,9.5,0.5,0.5,2.70,0.0,0.3,13.50
3,135,2,2015-01-07 14:58:07,2015-01-07 15:10:17,2,1.58,-73.981956,40.769260,1,N,-73.973465,40.789776,1,9.5,0.0,0.5,1.00,0.0,0.3,11.30
4,206,1,2015-01-26 13:04:18,2015-01-26 13:15:21,1,2.10,-73.944443,40.741562,1,N,-73.931107,40.764935,1,9.5,0.0,0.5,1.00,0.0,0.3,11.30


In [57]:
io.sql.read_sql_query("SELECT * FROM yellow WHERE DATE(tpep_pickup_datetime) = '2015-01-15'",engine).head()

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.00,0.3,17.05
1,15,2,2015-01-15 19:05:39,2015-01-15 19:32:00,1,2.38,-73.976425,40.739811,1,N,-73.983978,40.757889,1,16.5,1.0,0.5,4.38,0.00,0.3,22.68
2,16,2,2015-01-15 19:05:40,2015-01-15 19:21:00,5,2.83,-73.968704,40.754246,1,N,-73.955124,40.786858,2,12.5,1.0,0.5,0.00,0.00,0.3,14.30
3,17,2,2015-01-15 19:05:40,2015-01-15 19:28:18,5,8.33,-73.863060,40.769581,1,N,-73.952713,40.785782,1,26.0,1.0,0.5,8.08,5.33,0.3,41.21
4,18,2,2015-01-15 19:05:41,2015-01-15 19:20:36,1,2.37,-73.945541,40.779423,1,N,-73.980850,40.786083,1,11.5,1.0,0.5,0.00,0.00,0.3,13.30
